In [1]:
import sys 
# reload(sys) 
# sys.setdefaultencoding('utf-8')
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import xgboost as xgb
import random
from featwheel.utils import NgramUtil,DistanceUtil, LogUtil, MathUtil
import argparse

import time

import torch
from torch.autograd import Variable
import torch.nn as nn
from models import NLINet
from mutils import get_optimizer
from data import get_nli, get_batch

from nltk.stem import SnowballStemmer
snowball_stemmer1 = SnowballStemmer('spanish')
snowball_stemmer2 = SnowballStemmer('english')
snowball_stemmer1.stem
snowball_stemmer2.stem

from nltk.corpus import stopwords
stops1 = set(stopwords.words("spanish"))
stops2 = set(stopwords.words("english"))


from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.cross_validation import cross_val_score

/Users/liushijing/anaconda3/envs/python27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Read Data

In [2]:
df_train_en_sp = pd.read_csv('./input/cikm_english_train_20180516.txt',sep='	', header=None,error_bad_lines=False)
df_train_sp_en = pd.read_csv('./input/cikm_spanish_train_20180516.txt',sep='	', header=None,error_bad_lines=False)
df_train_en_sp.columns = ['english1', 'spanish1', 'english2', 'spanish2', 'result']
df_train_sp_en.columns = ['spanish1', 'english1', 'english2', 'spanish2', 'result']

# Evaluation data
test_data = pd.read_csv('./input/cikm_test_a_20180516.txt', sep='	', header=None,error_bad_lines=False)
test_data.columns = ['spanish1', 'spanish2']

In [3]:
train1 = pd.DataFrame(pd.concat([df_train_en_sp['spanish1'],df_train_sp_en['spanish1']], axis=0))
train2 = pd.DataFrame(pd.concat([df_train_en_sp['spanish2'],df_train_sp_en['spanish2']], axis=0))
train = pd.concat([train1,train2],axis=1).reset_index()
train = train.drop(['index'],axis=1)
result = pd.DataFrame(pd.concat([df_train_en_sp['result'],df_train_sp_en['result']], axis=0)).reset_index()
result = result.drop(['index'],axis=1)
# pd.get_dummies(result['result']).head()
train['result'] = result
train.head()

,spanish1,spanish2,result
0,"hola, hago clic en el producto recibido",Compré un producto y no he recibido un correo ...,0
1,¡Hola! Cerré la disputa el 21 de mayo de 2017 ...,No obtuve el reembolso de mi dinero. Han pasad...,0
2,Ordené de España a España ahora que mandan el ...,Mi pedido llegó pero el color es diferente al ...,0
3,¿Debo pagar impuestos personalizados?,Cómo pagar los derechos de aduana,1
4,No recibí mi pedido?,"Mi pedido muestra que no he pagado, pero lo hice",0


## Clean Text

In [4]:
def subchar(text):
	text=text.replace("á", "a")
	text=text.replace("ó", "o")
	text=text.replace("é", "e")	
	text=text.replace("í", "i")
	text=text.replace("ú", "u")	
	return text

In [5]:
#西班牙语缩写还原#
def stemSpanish(df):
	df['spanish1'] = df.spanish1.map(lambda x: ' '.join([snowball_stemmer1.stem(word) for word in 
                                                         nltk.word_tokenize(x.lower().decode('utf-8'))]).encode('utf-8'))
	df['spanish2'] = df.spanish2.map(lambda x: ' '.join([snowball_stemmer1.stem(word) for word in 
                                                         nltk.word_tokenize(x.lower().decode('utf-8'))]).encode('utf-8'))
def stemEnglish(df):
    df['english1'] =  df.english1.map(lambda x:' '.join([snowball_stemmer2.stem(word) for word in 
                                                        nltk.word_tokenize(x.lower().decode('utf-8'))]).encode('utf-8'))
    df['english2'] =  df.english2.map(lambda x:' '.join([snowball_stemmer2.stem(word) for word in 
                                                        nltk.word_tokenize(x.lower().decode('utf-8'))]).encode('utf-8'))

In [6]:
def removeSpanishStopWords(df, stop):
	df['spanish1'] = df.spanish1.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x.decode('utf-8'))
                                                         if word not in stops1]).encode('utf-8'))
	df['spanish2'] = df.spanish2.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x.decode('utf-8'))
                                                         if word not in stops1]).encode('utf-8'))
    
def removeEnglishStopWords(df, stop):
	df['english1'] = df.english1.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x.decode('utf-8'))
                                                         if word not in stops2]).encode('utf-8'))
	df['english2'] = df.english2.map(lambda x: ' '.join([word for word in nltk.word_tokenize(x.decode('utf-8'))
                                                         if word not in stops2]).encode('utf-8'))
    
def removeEnglishSigns(df):
    df['english1'] = df.english1.map(lambda x: re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]','',x))
    df['english2'] = df.english2.map(lambda x: re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]','',x))
def removeSpanishSigns(df):
    df['spanish1'] = df.spanish1.map(lambda x: re.sub(r'[_"\-;%()|+&=*%.,¡!¿?:#$@\[\]/]','',x))
    df['spanish2'] = df.spanish2.map(lambda x: re.sub(r'[_"\-;%()|+&=*%.,¡!¿?:#$@\[\]/]','',x))

In [7]:
train.head()

,spanish1,spanish2,result
0,"hola, hago clic en el producto recibido",Compré un producto y no he recibido un correo ...,0
1,¡Hola! Cerré la disputa el 21 de mayo de 2017 ...,No obtuve el reembolso de mi dinero. Han pasad...,0
2,Ordené de España a España ahora que mandan el ...,Mi pedido llegó pero el color es diferente al ...,0
3,¿Debo pagar impuestos personalizados?,Cómo pagar los derechos de aduana,1
4,No recibí mi pedido?,"Mi pedido muestra que no he pagado, pero lo hice",0


In [8]:
# 处理 Spanish
stemSpanish(train)
removeSpanishSigns(train)

In [9]:
train.head()

,spanish1,spanish2,result
0,hol hag clic en el product recib,compr un product y no he recib un corre electr...,0
1,hol cerr la disput el 21 de may de 2017 y dic...,no obtuv el reembols de mi diner han pas dos ...,0
2,orden de españ a españ ahor que mand el ped de...,mi ped lleg per el color es diferent al que ped,0
3,deb pag impuest personaliz,com pag los derech de aduan,1
4,no recib mi ped,mi ped muestr que no he pag per lo hic,0


### Clean evaluation set

In [10]:
stemSpanish(test_data)
removeSpanishSigns(test_data)

### Split Train, Dev, Test

In [11]:
train_data = train
train_data.columns = ['s1','s2','label']
test_data.columns = ['s1','s2']

In [12]:
# 按比例分割 train dev test
total_len = train_data.shape[0]
train_len = int(total_len * 0.8)
dev_len = train_len + int(total_len * 0.15)
test_len = dev_len + int(total_len * 0.05)

train = train_data.loc[0:train_len-1]
valid = train_data.loc[train_len:dev_len-1]
test = train_data.loc[dev_len:]

In [13]:
# W2V_PATH = "input/wiki.es.vec"

In [14]:
# word_vec = build_vocab(train['s1'].tolist() + train['s2'].tolist() +
#                        valid['s1'].tolist() + valid['s2'].tolist() +
#                        test['s1'].tolist() + test['s2'].tolist(), W2V_PATH)

In [15]:
# for split in ['s1', 's2']:
#     for data_type in ['train', 'valid', 'test']:
#         eval(data_type)[split] = np.array([['<s>'] +
#             [word for word in sent.split() if word in word_vec] +
#             ['</s>'] for sent in eval(data_type)[split]])

## Embedding Matrix

In [16]:
# from gensim.models import KeyedVectors
# es_vec = KeyedVectors.load_word2vec_format('input/wiki.es.vec') 

### Vocab

In [17]:
def get_word_dict(sentences):
    # create vocab of words
    word_dict = {}
    for sent in sentences:
        for word in sent.split():
            if word not in word_dict:
                word_dict[word] = ''
    word_dict['<s>'] = ''
    word_dict['</s>'] = ''
    word_dict['<p>'] = ''
    word_dict['<unk>'] = ''
    return word_dict
all_sent = train_data['s1'].tolist() + train_data['s2'].tolist() + test_data['s1'].tolist() + test_data['s2'].tolist()
word_dict = get_word_dict(all_sent)

### Embedding vectors

In [18]:
def get_embedding(word_dict, embedding_path):
    # create word_vec with embedding vectors
    word_vec = {}
    with open(embedding_path) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word in word_dict:
                word_vec[word] = np.array(list(map(float, vec.split())))
    print('Found {0}/{1} words with embedding vectors'.format(
                len(word_vec), len(word_dict)))
    missing_word_num = len(word_dict) - len(word_vec)
    missing_ratio = round(float(missing_word_num)/len(word_dict),4)*100
    print('Missing Ratio: {}%'.format(missing_ratio))
    return word_vec

In [19]:
embedding_path = 'input/wiki.es.vec'
word_vec = get_embedding(word_dict, embedding_path)

Found 2403/3894 words with embedding vectors
Missing Ratio: 38.29%


### Token

In [20]:
vocab_to_int = {}
value = 0
for word in word_dict:
    vocab_to_int[word] = value
    value += 1
        
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

### Handling unknown words' embeddings

In [25]:
embedding_dim = 300
for word, i in vocab_to_int.items():
    if word not in word_vec:
        # If word not in word_vec, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        word_vec[word] = new_embedding
print "Filled Unk words' embeddings"

Filled Unk words' embeddings


In [39]:
# # Need to use 300 for embedding dimensions to match given vectors.
# embedding_dim = 300
# nb_words = len(vocab_to_int)

# # Create matrix with default values of zero
# word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
# for word, i in vocab_to_int.items():
#     if word in word_vec:
#         word_embedding_matrix[i] = word_vec[word]
#     else:
#         # If word not in word_vec, create a random embedding for it
#         new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
#         word_vec[word] = new_embedding
#         word_embedding_matrix[i] = new_embedding

# # Check if value matches len(vocab_to_int)
# print(len(word_embedding_matrix))

### Tokenize sentences

In [40]:
for split in ['s1', 's2']:
    for data_type in ['train', 'valid', 'test']:
        eval(data_type)[split] = np.array([['<s>'] +
            [word for word in sent.split() if word in word_vec] +
            ['</s>'] for sent in eval(data_type)[split].tolist()])

AttributeError: 'list' object has no attribute 'split'

In [21]:
# def convert_to_ints(text):
#     '''Convert words in text to an integer.
#        If word is not in vocab_to_int, use UNK's integer.
#        Total the number of words and UNKs.
#        Add EOS token to the end of texts'''
#     ints = []
#     unk_count = 0
#     for sentence in text:
#         sentence_ints = [vocab_to_int['<s>']]
#         for word in sentence.split():
#             if word in vocab_to_int:
#                 sentence_ints.append(vocab_to_int[word])
#             else:
#                 sentence_ints.append(vocab_to_int['<unk>'])
#                 unk_count += 1
#             sentence_ints.append(vocab_to_int['</s>'])
#         ints.append(sentence_ints)
#     return ints, unk_count

In [22]:
# tokenize_sent,unk_count = convert_to_ints(all_sent)

## Model

### Params

In [41]:
parser = argparse.ArgumentParser(description='InferSent training')
# paths
parser.add_argument("--outputdir", type=str, default='savedir/', help="Output directory")
parser.add_argument("--outputmodelname", type=str, default='model.pickle')


# training
parser.add_argument("--n_epochs", type=int, default=20)
parser.add_argument("--batch_size", type=int, default=64)
parser.add_argument("--dpout_model", type=float, default=0., help="encoder dropout")
parser.add_argument("--dpout_fc", type=float, default=0., help="classifier dropout")
parser.add_argument("--nonlinear_fc", type=float, default=0, help="use nonlinearity in fc")
parser.add_argument("--optimizer", type=str, default="sgd,lr=0.1", help="adam or sgd,lr=0.1")
parser.add_argument("--lrshrink", type=float, default=5, help="shrink factor for sgd")
parser.add_argument("--decay", type=float, default=0.99, help="lr decay")
parser.add_argument("--minlr", type=float, default=1e-5, help="minimum lr")
parser.add_argument("--max_norm", type=float, default=5., help="max norm (grad clipping)")

# model
parser.add_argument("--encoder_type", type=str, default='InferSent', help="see list of encoders")
parser.add_argument("--enc_lstm_dim", type=int, default=2048, help="encoder nhid dimension")
parser.add_argument("--n_enc_layers", type=int, default=1, help="encoder num layers")
parser.add_argument("--fc_dim", type=int, default=512, help="nhid of fc layers")
parser.add_argument("--n_classes", type=int, default=3, help="entailment/neutral/contradiction")
parser.add_argument("--pool_type", type=str, default='max', help="max or mean")
params, _ = parser.parse_known_args()
params.word_emb_dim = 300

In [42]:
print params

Namespace(batch_size=64, decay=0.99, dpout_fc=0.0, dpout_model=0.0, enc_lstm_dim=2048, encoder_type='InferSent', fc_dim=512, lrshrink=5, max_norm=5.0, minlr=1e-05, n_classes=3, n_enc_layers=1, n_epochs=20, nonlinear_fc=0, optimizer='sgd,lr=0.1', outputdir='savedir/', outputmodelname='model.pickle', pool_type='max', word_emb_dim=300)


In [43]:
# model config
config_nli_model = {
    'n_words'        :  len(word_vec)          ,
    'word_emb_dim'   :  params.word_emb_dim   ,
    'enc_lstm_dim'   :  params.enc_lstm_dim   ,
    'n_enc_layers'   :  params.n_enc_layers   ,
    'dpout_model'    :  params.dpout_model    ,
    'dpout_fc'       :  params.dpout_fc       ,
    'fc_dim'         :  params.fc_dim         ,
    'bsize'          :  params.batch_size     ,
    'n_classes'      :  params.n_classes      ,
    'pool_type'      :  params.pool_type      ,
    'nonlinear_fc'   :  params.nonlinear_fc   ,
    'encoder_type'   :  params.encoder_type   ,
    'use_cuda'       :  True                  ,

}

# model
encoder_types = ['InferSent', 'BLSTMprojEncoder', 'BGRUlastEncoder',
                 'InnerAttentionMILAEncoder', 'InnerAttentionYANGEncoder',
                 'InnerAttentionNAACLEncoder', 'ConvNetEncoder', 'LSTMEncoder']
assert params.encoder_type in encoder_types, "encoder_type must be in " + \
                                             str(encoder_types)
nli_net = NLINet(config_nli_model)
print(nli_net)

# loss
weight = torch.FloatTensor(params.n_classes).fill_(1)
loss_fn = nn.CrossEntropyLoss(weight=weight)
loss_fn.size_average = False

# optimizer
optim_fn, optim_params = get_optimizer(params.optimizer)
optimizer = optim_fn(nli_net.parameters(), **optim_params)

NLINet(
  (encoder): InferSent(
    (enc_lstm): LSTM(300, 2048, bidirectional=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=16384, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=3, bias=True)
  )
)


## Train

In [44]:
"""
TRAIN
"""
val_acc_best = -1e10
adam_stop = False
stop_training = False
lr = optim_params['lr'] if 'sgd' in params.optimizer else None

In [45]:
def trainepoch(epoch):
    print('\nTRAINING : Epoch ' + str(epoch))
    nli_net.train()
    all_costs = []
    logs = []
    words_count = 0

    last_time = time.time()
    correct = 0.
    # shuffle the data
    permutation = np.random.permutation(len(train['s1']))

    s1 = train['s1'][permutation]
    s2 = train['s2'][permutation]
    target = train['label'][permutation]


    optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * params.decay if epoch>1\
        and 'sgd' in params.optimizer else optimizer.param_groups[0]['lr']
    print('Learning rate : {0}'.format(optimizer.param_groups[0]['lr']))

    for stidx in range(0, len(s1), params.batch_size):
        # prepare batch
        s1_batch, s1_len = get_batch(s1[stidx:stidx + params.batch_size].tolist(),
                                     word_vec)
        s2_batch, s2_len = get_batch(s2[stidx:stidx + params.batch_size].tolist(),
                                     word_vec)
        s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
        tgt_batch = Variable(torch.LongTensor(target[stidx:stidx + params.batch_size]))
        k = s1_batch.size(1)  # actual batch size

        # model forward
        output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))

        pred = output.data.max(1)[1]
        correct += pred.long().eq(tgt_batch.data.long()).cpu().sum()
        assert len(pred) == len(s1[stidx:stidx + params.batch_size])

        # loss
        loss = loss_fn(output, tgt_batch)
        all_costs.append(loss.data[0])
        words_count += (s1_batch.nelement() + s2_batch.nelement()) / params.word_emb_dim

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient clipping (off by default)
        shrink_factor = 1
        total_norm = 0

        for p in nli_net.parameters():
            if p.requires_grad:
                p.grad.data.div_(k)  # divide by the actual batch size
                total_norm += p.grad.data.norm() ** 2
        total_norm = np.sqrt(total_norm)

        if total_norm > params.max_norm:
            shrink_factor = params.max_norm / total_norm
        current_lr = optimizer.param_groups[0]['lr'] # current lr (no external "lr", for adam)
        optimizer.param_groups[0]['lr'] = current_lr * shrink_factor # just for update

        # optimizer step
        optimizer.step()
        optimizer.param_groups[0]['lr'] = current_lr

        if len(all_costs) == 100:
            logs.append('{0} ; loss {1} ; sentence/s {2} ; words/s {3} ; accuracy train : {4}'.format(
                            stidx, round(np.mean(all_costs), 2),
                            int(len(all_costs) * params.batch_size / (time.time() - last_time)),
                            int(words_count * 1.0 / (time.time() - last_time)),
                            round(100.*correct/(stidx+k), 2)))
            print(logs[-1])
            last_time = time.time()
            words_count = 0
            all_costs = []
    train_acc = round(100 * correct/len(s1), 2)
    print('results : epoch {0} ; mean accuracy train : {1}'
          .format(epoch, train_acc))
    return train_acc



NameError: name 's1' is not defined

In [46]:
epoch = 1

while not stop_training and epoch <= params.n_epochs:
    train_acc = trainepoch(epoch)
    eval_acc = evaluate(epoch, 'valid')
    epoch += 1


TRAINING : Epoch 1
Learning rate : 0.1


/Users/liushijing/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


KeyboardInterrupt: 

In [ ]:
def evaluate(epoch,eval_type='valid', final_eval=False):
    nli_net.eval()
    correct = 0.
    global val_acc_best, lr, stop_training, adam_stop

    if eval_type == 'valid':
        print('\nVALIDATION : Epoch {0}'.format(epoch))

    s1 = valid['s1'] if eval_type == 'valid' else test['s1']
    s2 = valid['s2'] if eval_type == 'valid' else test['s2']
    target = valid['label'] if eval_type == 'valid' else test['label']

    for i in range(0, len(s1), params.batch_size):
        # prepare batch
        s1_batch, s1_len = get_batch(s1[i:i + params.batch_size].tolist(), word_vec)
        s2_batch, s2_len = get_batch(s2[i:i + params.batch_size].tolist(), word_vec)
        s1_batch, s2_batch = Variable(s1_batch), Variable(s2_batch)
        tgt_batch = Variable(torch.LongTensor(target[i:i + params.batch_size]))

        # model forward
        output = nli_net((s1_batch, s1_len), (s2_batch, s2_len))

        pred = output.data.max(1)[1]
        correct += pred.long().eq(tgt_batch.data.long()).cpu().sum()

    # save model
    eval_acc = round(100 * correct / len(s1), 2)
    if final_eval:
        print('finalgrep : accuracy {0} : {1}'.format(eval_type, eval_acc))
    else:
        print('togrep : results : epoch {0} ; mean accuracy {1} :\
              {2}'.format(epoch, eval_type, eval_acc))

    if eval_type == 'valid' and epoch <= params.n_epochs:
        if eval_acc > val_acc_best:
            print('saving model at epoch {0}'.format(epoch))
            if not os.path.exists(params.outputdir):
                os.makedirs(params.outputdir)
            torch.save(nli_net.state_dict(), os.path.join(params.outputdir,
                       params.outputmodelname))
            val_acc_best = eval_acc
        else:
            if 'sgd' in params.optimizer:
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] / params.lrshrink
                print('Shrinking lr by : {0}. New lr = {1}'
                      .format(params.lrshrink,
                              optimizer.param_groups[0]['lr']))
                if optimizer.param_groups[0]['lr'] < params.minlr:
                    stop_training = True
            if 'adam' in params.optimizer:
                # early stopping (at 2nd decrease in accuracy)
                stop_training = adam_stop
                adam_stop = True
    return eval_acc